# Logistic Regression Modelling: Predicting Cardiovascular Disease

<a id="1"></a>
## 🔍 1. Introduction

The World Health Organization has estimated 12 million deaths occur worldwide, every year due to Heart diseases. Half the deaths in the United States and other developed countries are due to cardio vascular diseases. The early prognosis of cardiovascular diseases can aid in making decisions on lifestyle changes in high risk patients and in turn reduce the complications. This research intends to pinpoint the most relevant/risk factors of heart disease as well as predict the overall risk using logistic regression.

**Goal:** <br>
The goal is to whether patient have 10 year risk of coronary heart disease CHD or not.

**Data source:**<br>
The dataset is publically available on the [Kaggle website](https://www.kaggle.com/datasets/christofel04/cardiovascular-study-dataset-predict-heart-disea/data) and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The dataset provides the patients’ information. It includes over 4,000 records and 15 attributes.

**Variables:**<br>
- **age:** patient age<br>
- **education:** Some high school (1), high school/GED (2), some college/vocational school (3), college (4)<br>
- **sex:** male (M), female (F)<br>
- **is_smoking:** whether or not the patient is a current smoker ("YES" or "NO")<br>
- **cigsPerDay:** the number of cigarettes that the person smoked on average in one day (can be considered continuous as one can have any number of cigarettes, even half a cigarette.)<br>
- **BPMeds:** whether or not the patient was on blood pressure medication (nominal)<br>
- **prevalentStroke:** whether or not the patient had previously had a stroke (nominal)<br>
- **prevalentHyp:** whether or not the patient was hypertensive (nominal)<br>
- **diabetes:** whether or not the patient had diabetes (nominal)<br>
- **totChol:** total cholesterol level (continuous)<br>
- **sysBP:** systolic blood pressure (continuous)<br>
- **diaBP:** diastolic blood pressure (continuous)<br>
- **BMI:** Body Mass Index (continuous)<br>
- **heartRate:** heart rate (continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)<br>
- **glucose:** glucose level (continuous)<br>
- **TenYearCHD:** predict variable; 10-xear risk of coronary heart disease CHD; binary: “1”, means “Yes”, “0” means “No”


**Project outline:**<br>
1. Introduction
2. Data loading & cleaning
3. EDA 
4. Modelling





<a id="2"></a>
## 🧹 2. Data loading & cleaning

In [75]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.feature_selection import f_classif

In [76]:
df_train = pd.read_csv("train.csv", index_col=0)
df_train.shape

(3390, 16)

In [77]:
df_train.head()

,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
id,,,,,,,,,,,,,,,,
0,64,2.0,F,YES,3.0,0.0,0,0,0,221.0,148.0,85.0,NaN,90.0,80.0,1
1,36,4.0,M,NO,0.0,0.0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0
2,46,1.0,F,YES,10.0,0.0,0,0,0,250.0,116.0,71.0,20.35,88.0,94.0,0
3,50,1.0,M,YES,20.0,0.0,0,1,0,233.0,158.0,88.0,28.26,68.0,94.0,1
4,64,1.0,F,YES,30.0,0.0,0,0,0,241.0,136.5,85.0,26.42,70.0,77.0,0


In [78]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3390 entries, 0 to 3389
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              3390 non-null   int64  
 1   education        3303 non-null   float64
 2   sex              3390 non-null   object 
 3   is_smoking       3390 non-null   object 
 4   cigsPerDay       3368 non-null   float64
 5   BPMeds           3346 non-null   float64
 6   prevalentStroke  3390 non-null   int64  
 7   prevalentHyp     3390 non-null   int64  
 8   diabetes         3390 non-null   int64  
 9   totChol          3352 non-null   float64
 10  sysBP            3390 non-null   float64
 11  diaBP            3390 non-null   float64
 12  BMI              3376 non-null   float64
 13  heartRate        3389 non-null   float64
 14  glucose          3086 non-null   float64
 15  TenYearCHD       3390 non-null   int64  
dtypes: float64(9), int64(5), object(2)
memory usage: 450.2+ KB


When checking the datatypes it seems that some variables are floats which do not need to be floats, while others are integers. For consistency purposes, we'll transform these. 

Looking at missing values, some variables do have missing values and we'll need to decide how to handle these. First, we'll check the share of missing data in each column.

In [79]:
df_train.isna().mean()

age                0.000000
education          0.025664
sex                0.000000
is_smoking         0.000000
cigsPerDay         0.006490
BPMeds             0.012979
prevalentStroke    0.000000
prevalentHyp       0.000000
diabetes           0.000000
totChol            0.011209
sysBP              0.000000
diaBP              0.000000
BMI                0.004130
heartRate          0.000295
glucose            0.089676
TenYearCHD         0.000000
dtype: float64

The variable we aim to predict (TenYearCHD) has no missing values which is a good thing. For other variables, the proportian of missing values is maximum 8% (glucose). Let's therefore see how much data is 'lost' if we drop all missign data.  

In [80]:
df_clean = df_train.dropna()
df_clean.shape

(2927, 16)

After removing all data with missing values, we are left with 2927 rows. 14% of the dataset were removed. For this project, we'll accept this 'dataloss' in order to work with clean and complete data going forward. 

In [81]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2927 entries, 1 to 3389
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              2927 non-null   int64  
 1   education        2927 non-null   float64
 2   sex              2927 non-null   object 
 3   is_smoking       2927 non-null   object 
 4   cigsPerDay       2927 non-null   float64
 5   BPMeds           2927 non-null   float64
 6   prevalentStroke  2927 non-null   int64  
 7   prevalentHyp     2927 non-null   int64  
 8   diabetes         2927 non-null   int64  
 9   totChol          2927 non-null   float64
 10  sysBP            2927 non-null   float64
 11  diaBP            2927 non-null   float64
 12  BMI              2927 non-null   float64
 13  heartRate        2927 non-null   float64
 14  glucose          2927 non-null   float64
 15  TenYearCHD       2927 non-null   int64  
dtypes: float64(9), int64(5), object(2)
memory usage: 388.7+ KB


We now have no missing values in the dataset. 

In [82]:
cols_to_convert = ['education', 'cigsPerDay', 'BPMeds', 'totChol', 'heartRate', 'glucose']

df_clean[cols_to_convert] = df_clean[cols_to_convert].astype('int64')

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2927 entries, 1 to 3389
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              2927 non-null   int64  
 1   education        2927 non-null   int64  
 2   sex              2927 non-null   object 
 3   is_smoking       2927 non-null   object 
 4   cigsPerDay       2927 non-null   int64  
 5   BPMeds           2927 non-null   int64  
 6   prevalentStroke  2927 non-null   int64  
 7   prevalentHyp     2927 non-null   int64  
 8   diabetes         2927 non-null   int64  
 9   totChol          2927 non-null   int64  
 10  sysBP            2927 non-null   float64
 11  diaBP            2927 non-null   float64
 12  BMI              2927 non-null   float64
 13  heartRate        2927 non-null   int64  
 14  glucose          2927 non-null   int64  
 15  TenYearCHD       2927 non-null   int64  
dtypes: float64(3), int64(11), object(2)
memory usage: 388.7+ KB


/var/folders/j6/b6gh915d113fbv3bgzvphpj40000gn/T/ipykernel_29799/2993514037.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



We now also have datatypes of all variables transformed for a coherent view. 

In [83]:
df_clean[df_clean.duplicated()]


,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
id,,,,,,,,,,,,,,,,


We have no instances of duplicated data.

In [84]:
df_clean.describe().round(2)

,age,education,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00,2927.00
mean,49.51,1.97,9.11,0.03,0.01,0.31,0.03,237.13,132.63,82.91,25.80,75.89,81.93,0.15
std,8.60,1.02,11.88,0.17,0.08,0.46,0.16,44.61,22.33,12.08,4.13,11.97,24.11,0.36
min,32.00,1.00,0.00,0.00,0.00,0.00,0.00,113.00,83.50,48.00,15.96,45.00,40.00,0.00
25%,42.00,1.00,0.00,0.00,0.00,0.00,0.00,206.00,117.00,74.50,23.03,68.00,71.00,0.00
50%,49.00,2.00,0.00,0.00,0.00,0.00,0.00,234.00,128.50,82.00,25.38,75.00,78.00,0.00
75%,56.00,3.00,20.00,0.00,0.00,1.00,0.00,264.00,144.00,90.00,28.04,83.00,87.00,0.00
max,70.00,4.00,70.00,1.00,1.00,1.00,1.00,600.00,295.00,142.50,56.80,143.00,394.00,1.00


Looking at the basic statistics for numeric variables, all seem generally reasonable. 
- age ranges from 32 to 70
- cigsPerDay has a max value of 70, which does seem high but still possible
- totChol with a max of 600(mg/dL) is in the very high range but not impossible
- Similar, a BMI of 56.80 is also very high and considered severely obese, but not impossible. 

Let's now also look at the categorical variables. 

In [85]:
df_clean['sex'].value_counts()

sex
F    1620
M    1307
Name: count, dtype: int64

In [86]:
df_clean['is_smoking'].value_counts()

is_smoking
NO     1480
YES    1447
Name: count, dtype: int64

Categorical variables also look fine and without any more need of cleaning.

<a id="3"></a>
## 🧮 3. EDA

We'll start with basic data exploration for the dataset and variables, especially focusing on the target variable (TenYearCHD) and how other variables relate to it.

In [87]:
px.histogram(df_clean, x='TenYearCHD',
             title='Heart Disease cases distribution')

Looking at the target variable only, we can see that we have an imbalance of cases in the dataset for the target variable. This is good to keep in mind and will be important later when we look at the accuracy, precision and recall of our model. 

In [88]:
px.histogram(df_clean, x='sex', 
             title='Sex vs. Heart Disease', 
             color='TenYearCHD')

Our dataset contains more data on females (57%) than males (43%). The proportiens of patients having heart disease seems quite similar between genders, although the share with heart disease is slightly higher for men:
- Men: 18% with heart disease
- Women: 13% with heart disease

In [89]:
px.box(df_clean, y='age', 
       title='Age vs. Heart Disease', 
       x='TenYearCHD')

Patients with heart disease clearly have a higher mean and median age than patients with no heart disease. Age seems to intuitively be a more significant factor than gender. 

In [90]:
px.histogram(df_clean, x='education', 
       title='Eduation vs. Heart Disease', 
       color='TenYearCHD')

The share of patients with heart disease seems highest for 'some high school' education (1) and then decreasing with higher education. Important to note that this is not a causal relationship (low education causing heart disease) but potentially still a correlation present between education and heart disease. 

In [91]:
px.histogram(df_clean, x='is_smoking', 
       title='Smoking vs. Heart Disease', 
       color='TenYearCHD')

Interestingly, the difference between whether a patient is a smoker or not, and having heart disease, is not as distinct as expected. However, smokers still seem to have a higher share of patients with heart disease.

In [92]:
px.histogram(df_clean, x='BPMeds', 
       title='Blood Pressure Medication vs. Heart Disease', 
       color='TenYearCHD')

The sample for patients with blood presure medication is small, but seemingly a higher share of those having heart disease (34%) vs. those without the medication (15%).

In [93]:
px.histogram(df_clean, x='prevalentStroke', 
       title='Prevalent Stroke vs. Heart Disease', 
       color='TenYearCHD')

Also here, a very small sample of patients with prevalent strokes, but of those, all having heart disease.

In [94]:
px.histogram(df_clean, x='prevalentHyp', 
       title='Hypertensive vs. Heart Disease', 
       color='TenYearCHD')

24% of patients with hypertensitivity have heart disease, while 11% of patients not being hypertensive have heart disease which likely is a statistically significant difference. 

Let's also look at the correlations between continuous variables in the datset.

In [95]:
continuous_var = ['cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']
fig = px.scatter_matrix(df_clean,
                        dimensions=continuous_var,
                        color='TenYearCHD')
fig.update_layout(height=1000)
fig.show()

In [96]:
df_clean[continuous_var].corr()

,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose
cigsPerDay,1.000000,-0.026606,-0.105104,-0.067483,-0.095310,0.055770,-0.064029
totChol,-0.026606,1.000000,0.210685,0.165101,0.113206,0.091487,0.066703
sysBP,-0.105104,0.210685,1.000000,0.783586,0.340782,0.180690,0.134609
diaBP,-0.067483,0.165101,0.783586,1.000000,0.391291,0.173350,0.074630
BMI,-0.095310,0.113206,0.340782,0.391291,1.000000,0.076848,0.091094
heartRate,0.055770,0.091487,0.180690,0.173350,0.076848,1.000000,0.089920
glucose,-0.064029,0.066703,0.134609,0.074630,0.091094,0.089920,1.000000


Looking at the scatter plots it seems that we only have one clear and strong correlation between sysBP (systolic blood pressure) and diaBP (diastolic blood pressure). This is not surprising as both values are interlinked and defined as: <br>

_"Systolic pressure is the maximum blood pressure during contraction of the ventricles; diastolic pressure is the minimum pressure recorded just prior to the next contraction."_

From the correlation chart we see that sysBP and diaBP in fact have a correlation coefficient of 0.78. Besides that, no other clear correlations between continuous variables can be seen. **Medium to low positive correlations** can be found between:
- sysBP and totChol (blood pressure and cholesterol level): 0.21
- sysBP and BMI (blood pressure and body mass index): 0.34
- diaBP and BMI (blood pressure and body mass index): 0.39


Also looking at the **predicted variable** of having heart disease or not (depicted by the color) does not show any clear patterns on first sight. One potential pattern or hypothesis that may be seen is that paients with heart disease (yellow) seem to have more outliers for higher glucose levels than patients without heart disease.  

<a id="4"></a>
## 📈 4. Modelling

In [97]:
numeric_cols = df_clean.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df_clean.select_dtypes('object').columns.tolist()

Before starting the modelling, we'll scale all numeric variables in order to give them the same 'weight' and not have a specific variables disproportianlly affect the loss of the model. 

In [98]:
scaler = MinMaxScaler()
scaler.fit(df_clean[numeric_cols])
df_clean[numeric_cols] = scaler.transform(df_clean[numeric_cols])
df_clean[numeric_cols].describe()

/var/folders/j6/b6gh915d113fbv3bgzvphpj40000gn/T/ipykernel_29799/902696585.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,age,education,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000
mean,0.460720,0.321831,0.130182,0.030065,0.006150,0.314315,0.026990,0.254885,0.232276,0.369378,0.240868,0.315187,0.118459,0.151691
std,0.226242,0.339066,0.169754,0.170795,0.078192,0.464322,0.162082,0.091608,0.105561,0.127819,0.101136,0.122155,0.068099,0.358783
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.263158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190965,0.158392,0.280423,0.173115,0.234694,0.087571,0.000000
50%,0.447368,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.248460,0.212766,0.359788,0.230656,0.306122,0.107345,0.000000
75%,0.631579,0.666667,0.285714,0.000000,0.000000,1.000000,0.000000,0.310062,0.286052,0.444444,0.295788,0.387755,0.132768,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


We can see that all numeric columns are now scaled to be within the range of 0 and 1. 

As a next step, we'll have to transform our categorical variables in order to be used in the modelling. 

In [99]:
df_clean[categorical_cols].nunique()

sex           2
is_smoking    2
dtype: int64

In [100]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(df_clean[categorical_cols])
encoder.categories_

[array(['F', 'M'], dtype=object), array(['NO', 'YES'], dtype=object)]

In the above we have 'fit' the encoder to our dataset and thereby identified all categories we have across our categorical columns. As a next step, we'll generate new column names for each of the categories.

In [101]:
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
print(encoded_cols)

['sex_F', 'sex_M', 'is_smoking_NO', 'is_smoking_YES']


The above columns will be added to our dataset through the below transformation steps.

In [102]:
df_clean[encoded_cols] = encoder.transform(df_clean[categorical_cols])
df_clean

/var/folders/j6/b6gh915d113fbv3bgzvphpj40000gn/T/ipykernel_29799/1964189330.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/j6/b6gh915d113fbv3bgzvphpj40000gn/T/ipykernel_29799/1964189330.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/j6/b6gh915d113fbv3bgzvphpj40000gn/T/ipykernel_29799/1964189330.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD,sex_F,sex_M,is_smoking_NO,is_smoking_YES
id,,,,,,,,,,,,,,,,,,,,
1,0.105263,1.000000,M,NO,0.000000,0.0,0.0,1.0,0.0,0.203285,0.399527,0.529101,0.338149,0.275510,0.098870,0.0,0.0,1.0,1.0,0.0
2,0.368421,0.000000,F,YES,0.142857,0.0,0.0,0.0,0.0,0.281314,0.153664,0.243386,0.107493,0.438776,0.152542,0.0,1.0,0.0,0.0,1.0
3,0.473684,0.000000,M,YES,0.285714,0.0,0.0,1.0,0.0,0.246407,0.352246,0.423280,0.301175,0.234694,0.152542,1.0,0.0,1.0,0.0,1.0
4,0.842105,0.000000,F,YES,0.428571,0.0,0.0,0.0,0.0,0.262834,0.250591,0.391534,0.256121,0.255102,0.104520,0.0,1.0,0.0,0.0,1.0
5,0.763158,0.666667,F,NO,0.000000,0.0,0.0,1.0,0.0,0.326489,0.465721,0.772487,0.412341,0.408163,0.070621,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,0.763158,0.000000,F,NO,0.000000,0.0,0.0,1.0,0.0,0.213552,0.465721,0.402116,0.269833,0.612245,0.206215,0.0,1.0,0.0,1.0,0.0
3385,0.736842,0.000000,F,NO,0.000000,0.0,0.0,0.0,0.0,0.303901,0.189125,0.328042,0.326151,0.255102,0.177966,0.0,1.0,0.0,1.0,0.0
3386,0.368421,0.000000,F,NO,0.000000,0.0,0.0,0.0,0.0,0.176591,0.087470,0.084656,0.146915,0.357143,0.124294,0.0,1.0,0.0,1.0,0.0


The new encoded columns are now added to the dataset.

In the next step, we split the dataset into train and test data. The training data will be used to train the machine learning model. The test data will be used to evaluate the model's performance on unseen data. 

In [103]:
X = df_clean[['age', 'education', 'BPMeds', 'cigsPerDay', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose', 'sex_F', 'sex_M', 'is_smoking_NO', 'is_smoking_YES']]
y = df_clean['TenYearCHD']

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

With the above function, we determine that 20% of the data goes to the test set, while the remainder is used for the training set. 

In the next step, we will fit the regression model to the training data.

In [105]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

After the model is fitted, we can look at the model coefficients below and which weight each input variable has been assigned.

In [106]:
print(model.coef_.tolist())

[[2.2536969290956783, -0.15138376189201003, 0.1102243080310863, 1.0881863939520298, 1.0298000956258413, 0.222062656055287, 0.2997049962690979, 0.9625413060409095, 2.0091761161440322, 0.286080096722086, 0.062430826244034704, -0.5155158590585103, 1.6694980265000479, -1.1741122650077436, -0.8275087524004856, -1.1050073398047953, -0.8966136776034201]]


In [107]:
# Creating a dataframe that contains all coefficients per feature

coefficients = model.coef_[0]

coef_df = pd.DataFrame({
    'feature': X_train.columns,
    'weight': coefficients
})

print(coef_df)

            feature    weight
0               age  2.253697
1         education -0.151384
2            BPMeds  0.110224
3        cigsPerDay  1.088186
4   prevalentStroke  1.029800
5      prevalentHyp  0.222063
6          diabetes  0.299705
7           totChol  0.962541
8             sysBP  2.009176
9             diaBP  0.286080
10              BMI  0.062431
11        heartRate -0.515516
12          glucose  1.669498
13            sex_F -1.174112
14            sex_M -0.827509
15    is_smoking_NO -1.105007
16   is_smoking_YES -0.896614


The higher the weight (coefficient above), the more weight is assigned for this variable on the model.<br>

Variables with the highest weight are age, sysBP and glucose while other variables have negative coefficients (education, diaBP, heartRate, sex and smoking)

In [108]:
px.bar(coef_df.sort_values('weight', ascending=True), 
       x='weight', 
       y='feature')

In [109]:
print(model.intercept_)


[-2.00162102]


The model intercept gives us the log-odds of the target variable if all features are valued at zero. A positive intercept suggests that the baseline odds of the target class are above 1, meaning the event is more likely than not when features are zero. A negative intercept implies that the baseline odds are below 1, indicating that the event is less likely when features are zero.

With our intercept being negative, this means that getting heart disease is less likely to occur when all features are zero. From the intercept we can also calculate the baseline probability. 

In [110]:
baseline_probability = 1 / (1 + np.exp(-model.intercept_))
print(baseline_probability)


[0.11903283]


This means that when all features are zero, there is a 12% chance of a patient getting heart disease. 

Now that we have fitted the model, it is time to evaluate the model fit with the test data.

In [111]:
train_pred = model.predict(X_train)

With the above code, we make predictions on the target variable based on the training data. Once we have the predictions, we can calculate the accuracy score of how these predictions match the actual target values of the train dataset.

In [112]:
accuracy_score(y_train, train_pred)

0.8568987612131568

The model achieves an **accuracy of 85.7%** for the training dataset. <br>Furthermore we can also get the result for the confusion matrix of the model. <br>
<br>
<img src="confusion-matrix.png" alt="confusion matrix" title="confusion matrix" width="500"/>

In [113]:
cm = confusion_matrix(y_train, train_pred, normalize='true')
cm

array([[0.99799096, 0.00200904],
       [0.94571429, 0.05428571]])

In [114]:
fig = px.imshow(cm,
                text_auto=True,
                color_continuous_scale='Blues',
                labels={'x': 'Prediction', 'y': 'Target'},
                title='Confusion Matrix Training')
fig.show()

The confusion matrix tells us the performance of our model, comparing the predicted values to the actual target values. 
<br>
- **True Negative (TN):** 99.8% - share of correctly identified negative cases. We have a high accuracy in identifying true negative cases. <br>
- **False Positive (FP):** 0.2% - share of negative cases that were incorrectly predicted as positive ('false alarms'). This means we have a high precision for the negative class and low share of falsely predicting negatives as positives. <br>
- **False Negative (FN):** 94.6% - share of positive cases that were incorrectly predicted as negative. This means the model is poor at identifying positive cases which is a big concerns for detecting heart disease. <br>
**True Positive (TP):** 5.4% - Only about 5% of positive cases were correctly identified as positive. This is definitely not good enough for a model aiming to detect heart disease. 
<br>
<br>
From the confusion matrix it becomes clear that our model is quite good in predicting the negative cases, while not very good in predicting the positive cases. 

In [115]:
precision = precision_score(y_train, train_pred)
recall = recall_score(y_train, train_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Precision: 0.8260869565217391
Recall: 0.054285714285714284


The **accuracy of the model** was high at 85.7% while we have seen that the model is not very good in predicting positive cases (having heart disease). Accuracy is high in this case as we have an imbalance of cases in our target variable (having heart disease vs. not having heart disease) and it is therefore not a reliable and good metric to look at to determine how good our model is. <br>
<br>
**Precision** tells us how often the model correctly predicts positive cases. Here we get 82.6% which is not bad. Precision is a better metric to use when we have an imbalance of cases and the cost of false positives is high.<br>
<br>
**Recall** tells us how good our model is at finding all the true positive cases. We here land at 5.4% which is very low. Recall is also a good metric to use for imbalanced cases and when the cost of false negatives is high. As we are working with data in the medical field and trying to predict whether a patient will have heart disease or not, the recall is the most important metric for us to look at and in this case not good enough. 

Before moving on to improve the model, we'll set up a function below that allows us to play around with different input data and adjusting the decision treshold.

In [116]:
def predict_and_plot(inputs, targets, model, threshold=0.5, name=''):
    # Get predicted probabilities for the positive class
    probs = model.predict_proba(inputs)[:, 1] 

    # Applying the threshold to get a binary prediction
    preds = (probs >= threshold).astype(int)
    
    accuracy = accuracy_score(targets, preds)
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    precision = precision_score(targets, preds)
    print("Precision: {:.2f}%".format(precision * 100))
    recall = recall_score(targets, preds)
    print("Recall: {:.2f}%".format(recall * 100))

    
    cf = confusion_matrix(targets, preds, normalize='true')
    fig = px.imshow(cf,
                text_auto=True,
                color_continuous_scale='Blues',
                labels={'x': 'Prediction', 'y': 'Target'},
                title=('{} Confusion Matrix'.format(name)))
    fig.show()


In [117]:
predict_and_plot(X_train, y_train, model, threshold=0.5, name='Train')

Accuracy: 85.69%
Precision: 82.61%
Recall: 5.43%


This is the training data we checked before. The default decision treshold is 0.5 and it therefore gives us the same results as before. We can now try this function for the test data.

In [118]:
predict_and_plot(X_test, y_test, model, threshold=0.5, name='Test')

Accuracy: 84.64%
Precision: 75.00%
Recall: 6.38%


On the test data, our recall is slightly better at 6.38%, while overall accuracy and precision have decreased. The recall however is still not good enough for a medical prediction model. We can therefore try lowering the decision treshold. 

In [119]:
predict_and_plot(X_train, y_train, model, threshold=0.1, name='Train 0.1 Treshold')

Accuracy: 50.88%
Precision: 21.43%
Recall: 85.71%


At a 0.1 decision treshold we get a recall of 85% which is much better. This now means that if the probability is above 10% for a patient having heart disease, it is now classified or predicted as positive. However, as we see, the overall accuracy and precision of the model decreases in this case. 

In [120]:
predict_and_plot(X_test, y_test, model, threshold=0.1, name='Test 0.1 Treshold')

Accuracy: 48.12%
Precision: 22.07%
Recall: 88.30%


For the test data, we get a higher recall of 88.3%. 

To further improve the model, we should also look at which variables could or should be excluded. To do this, we start with checking which of the numeric or continuous variables shows significant differences for the classes of the target variable through an ANOVA test.

In [121]:
f_values, p_values = f_classif(df_clean[numeric_cols], y)
anova_results = pd.DataFrame({
    'Feature': numeric_cols,
    'F-Statistic': f_values,
    'p-value': p_values.round(5)
})
anova_results = anova_results.sort_values('p-value').reset_index(drop=True)
print(anova_results)

            Feature  F-Statistic  p-value
0               age   170.087572  0.00000
1            BPMeds    25.445752  0.00000
2          diabetes    29.549166  0.00000
3      prevalentHyp    86.139230  0.00000
4           totChol    33.539563  0.00000
5        TenYearCHD          inf  0.00000
6             diaBP    57.631823  0.00000
7             sysBP   145.128664  0.00000
8           glucose    52.902934  0.00000
9               BMI    16.970465  0.00004
10        education    11.449326  0.00072
11       cigsPerDay    10.496486  0.00121
12  prevalentStroke     7.934838  0.00488
13        heartRate     1.028379  0.31062


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning:

divide by zero encountered in divide



From the ANOVA test we can see that almost all numeric or continuous variables show statistical significance (p < 0.05) against the target variable. For heartRate the p-value is above that treshold, indicating that there is not a strong association between heartRate and the target variable and it could therefore be excluded from the model. We'll run the model again with heartRate removed to see if it has any effect.

In [122]:

X = df_clean[['age', 'education', 'BPMeds', 'cigsPerDay', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'glucose', 'sex_F', 'sex_M', 'is_smoking_NO', 'is_smoking_YES']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

predict_and_plot(X_train, y_train, model, threshold=0.1, name='Train without heartRate')

Accuracy: 50.45%
Precision: 21.28%
Recall: 85.71%


Interestingly, the model achieves the same metric values with heartRate removed for a 0.1 treshold. heartRate indeed can therefore be removed from the model. 

We have also seen before that both sysBP and diaBP are strongly correlated which could lead to multicollinearity issue. We can therefore also try removing one of the blood pressure variables to understand the effect.

In [123]:

X = df_clean[['age', 'education', 'BPMeds', 'cigsPerDay', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'diaBP', 'BMI', 'glucose', 'sex_F', 'sex_M', 'is_smoking_NO', 'is_smoking_YES']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

predict_and_plot(X_train, y_train, model, threshold=0.1, name='Train without heartRate & sysBP')

Accuracy: 50.41%
Precision: 21.42%
Recall: 86.86%


Here we achieve the highest recall (86.86%) when removing sysBP.<br>

Let's see how this model performs on our test data.

In [125]:

X = df_clean[['age', 'education', 'BPMeds', 'cigsPerDay', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'diaBP', 'BMI', 'glucose', 'sex_F', 'sex_M', 'is_smoking_NO', 'is_smoking_YES']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LogisticRegression(solver='liblinear')
model.fit(X_test, y_test)

predict_and_plot(X_test, y_test, model, threshold=0.1, name='Test without heartRate & sysBP')

Accuracy: 48.46%
Precision: 22.34%
Recall: 89.36%


<a id="5"></a>
## 📝 5. Summary & Conclusions

Goal of this project was to fit a logistic regression model to a patient dataset in order to predict the 10-year risk of coronary heart disease (CHD). <br>

The model we fitted resulted in the below 'fit' metrics for a 0.1 decision treshold:<br>
- Accuracy: 48.46%<br>
- Precision: 22.34%<br>
- Recall: 89.36%<br>

The metric we optimized for in this model is **recall** since we make predictions for the medical field and the cost for false positives is high. We mostly care about finding all true positive cases which recall is the best metric for. On the flipside, this means we have to compromise on other metrics such as overall accuracy and precision. 

The outcome and prediction power of this model depends on what it actually will be used for. If the model is used to simply recommend patients to start a preventitive program based on their heart disease risk, 89% recall may be good enough. If it is used for something else where the stakes of being wrong are higher, this may not be good enough and we'd need to further improve the model. 